In [2]:
import pandas as pd
import numpy as np
import time

In [7]:
# Create a sample DataFrame
num_rows = 1000000  # Adjust the number of rows for testing
start_time = time.time()

df = pd.read_excel(r"20241119_BG_Matnumbers_ZRH.XLSX")
delta = time.time() - start_time 

print(delta)
# Define file names for different formats
file_names = {
    "Excel": "test_data.xlsx",
    'CSV': 'test_data.csv',
    'Parquet-pyarrow': 'test_data.parquet',
    'Parquet-fastparquet': 'test_data.parquet',
    'Feather': 'test_data.feather',
    'HDF5': 'test_data.h5'
}

0.18220186233520508


In [10]:
# Function to test read and write speeds
def test_speed(format_name, file_name):
    # Measure write speed
    start_time = time.time()
    if format_name == 'Excel':
        df.to_excel(file_name, index=False)
    if format_name == 'CSV':
        df.to_csv(file_name, index=False)
    elif format_name in {'Parquet-fastparquet', "Parquet-pyarrow"}:
        df.to_parquet(file_name, index=False)
    elif format_name == 'Feather':
        df.to_feather(file_name)
    elif format_name == 'HDF5':
        df.to_hdf(file_name, key='df', mode='w', index=False)
    write_time = time.time() - start_time

    # Measure read speed
    start_time = time.time()
    if format_name == 'CSV':
        pd.read_csv(file_name)
    elif format_name == 'Excel':
        pd.read_excel(file_name)
    elif format_name == 'Parquet-pyarrow':
        pd.read_parquet(file_name, engine="pyarrow")
    elif format_name == 'Parquet-fastparquet':
        pd.read_parquet(file_name, engine="fastparquet")
    elif format_name == 'Feather':
        pd.read_feather(file_name)
    elif format_name == 'HDF5':
        pd.read_hdf(file_name, key='df')
    read_time = time.time() - start_time

    return write_time, read_time

In [11]:
# Test each format
results = {}
for format_name, file_name in file_names.items():
    write_time, read_time = test_speed(format_name, file_name)
    results[format_name] = {'Write Time (s)': write_time, 'Read Time (s)': read_time}

# Print the results
for format_name, times in results.items():
    print(f"{format_name}: Write Time = {times['Write Time (s)']:.4f} s, Read Time = {times['Read Time (s)']:.4f} s")

results_df = pd.DataFrame(results).T  # Transpose to have formats as rows
results_df.index.name = 'Format'

# Print the results table
print("\n\n")
print(results_df)

Excel: Write Time = 0.4530 s, Read Time = 0.2995 s
CSV: Write Time = 0.0090 s, Read Time = 0.0188 s
Parquet-pyarrow: Write Time = 0.0104 s, Read Time = 0.0130 s
Parquet-fastparquet: Write Time = 0.0091 s, Read Time = 0.0112 s
Feather: Write Time = 0.0050 s, Read Time = 0.0121 s
HDF5: Write Time = 0.0122 s, Read Time = 0.0131 s



                     Write Time (s)  Read Time (s)
Format                                            
Excel                      0.453029       0.299455
CSV                        0.008998       0.018830
Parquet-pyarrow            0.010435       0.013000
Parquet-fastparquet        0.009116       0.011245
Feather                    0.005041       0.012136
HDF5                       0.012205       0.013144


In [8]:
df.to_csv("BG_Matnumbers_ZRH_final.csv", index=False)